<a href="https://colab.research.google.com/github/zayedupal/Hugging_Face_Movie_Genre_Prediction_Public/blob/main/HF_movie_genre_sentence_transformer_embedding_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers datasets evaluate accelerate torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 117.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 97.7 MB/s eta 0

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import time

import numpy as np
import pandas as pd
import torch.nn.functional as F
from datasets import load_dataset
from torch import Tensor
from tqdm import tqdm
from sentence_transformers import SentenceTransformer


def preprocess_dataset(data):
    def create_text(x):
        x['text'] = x['movie_name'] + ': ' + x['synopsis']
        x['label'] = x['genre']
        return x

    data = data.map(create_text)
    print(f"preprocessed data: {data}")
    return data


def generate_embeddings(input_texts, model):
    embeddings = model.encode(input_texts)

    return embeddings
    # return embeddings


def generate_embeddings_batch(input_texts, model, batch_size=50):
    embeddings = None
    batch_count = len(input_texts) // batch_size
    for i in tqdm(range(batch_count)):
        cur_embeddings = generate_embeddings(input_texts[i * batch_size: (i + 1) * batch_size], model)
        embeddings = np.append(embeddings, cur_embeddings, axis=0) if (embeddings is not None) else cur_embeddings
        del cur_embeddings
        # print(f"embedding completed: {i/batch_count}")

    if len(input_texts) > batch_count * batch_size:
        cur_embeddings = generate_embeddings(input_texts[batch_count * batch_size:], model)
        embeddings = np.append(embeddings, cur_embeddings, axis=0) if (embeddings is not None) else cur_embeddings
        del cur_embeddings

    print(f'len(embeddings): {len(embeddings)}')
    print(f'len(input_texts): {len(input_texts)}')

    return embeddings

def create_embedding_df(data, embeddings):
    print(f'embeddings: {embeddings.shape}')
    # print(f'embeddings flatten: {embeddings.flatten().shape}')
    embeddings_df = pd.DataFrame({'embeddings': embeddings.tolist()})
    embeddings_df['id'] = data['id']
    # embeddings_df['embeddings'] = embeddings
    embeddings_df['label'] = data['label']

    return embeddings_df

def generate_embedding_files_from_HF_dataset(model,
                                             dataset, train_emb_output, test_emb_output,
                                             train_split_percent=100, test_split_percent=100):
    train_data = load_dataset(dataset, use_auth_token=True, split=f'train[:{train_split_percent}%]')
    test_data = load_dataset(dataset, use_auth_token=True, split=f'test[:{test_split_percent}%]')

    print(f'train_data: {len(train_data)}')
    print(f'test_data: {len(test_data)}')

    train_data = preprocess_dataset(train_data)
    test_data = preprocess_dataset(test_data)

    train_embeddings = generate_embeddings_batch(train_data['text'], model)
    create_embedding_df(train_data, train_embeddings).to_csv(train_emb_output, index=False)
    del train_embeddings

    test_embeddings = generate_embeddings_batch(test_data['text'], model)
    create_embedding_df(test_data, test_embeddings).to_csv(test_emb_output, index=False)
    del test_embeddings

In [ ]:
# MAIN
start_time = time.time()

model_name = "sentence-transformers/gtr-t5-xxl"
formatted_model_name = model_name.replace("/","_")


model = SentenceTransformer(model_name)

generate_embedding_files_from_HF_dataset(
    model=model,
    dataset="datadrivenscience/movie-genre-prediction",
    train_emb_output=f"/content/drive/MyDrive/Colab Notebooks/train_embeddings_{formatted_model_name}.csv",
    test_emb_output=f"/content/drive/MyDrive/Colab Notebooks/test_embeddings_{formatted_model_name}.csv",
    train_split_percent=100,
    test_split_percent=100
)

print(f"Runtime: {time.time() - start_time}")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/54000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/datadrivenscience___parquet/datadrivenscience--movie-genre-prediction-01acd85570f2b187/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


train_data: 54000
test_data: 36000


...

preprocessed data: Dataset({
    features: ['id', 'movie_name', 'synopsis', 'genre', 'text', 'label'],
    num_rows: 36000
})


100%|██████████| 1080/1080 [36:42<00:00,  2.04s/it]


len(embeddings): 54000
len(input_texts): 54000
embeddings: (54000, 768)


100%|██████████| 720/720 [24:10<00:00,  2.02s/it]


len(embeddings): 36000
len(input_texts): 36000
embeddings: (36000, 768)
Runtime: 4428.202921390533
